<a href="https://colab.research.google.com/github/Sirabhop/MFP-Volunteer/blob/main/20220627_OPS_MOPH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import requests

In [ ]:
os.chdir('/content/drive/MyDrive/My Project/MFP_Volunteer/20220627_OPS_MOPH')

In [ ]:
#For tracking data
def track_status(df, compare_df, col_list):
  print('Row:', "{:,}".format(len(df.index)), 'and', 'Columns:', "{:,}".format(len(df.columns)))

  for i in range(len(col_list)):
    update_value = df[col_list[i]].sum()
    base_value = compare_df[col_list[i]].sum()
    print(col_list[i]+' of new df:', "{:,}".format(update_value))
    print(col_list[i]+' of base df:', "{:,}".format(base_value))
    if update_value == base_value:
      print('pass: the', col_list[i], 'are match')
    else:
      print(col_list[i], 'are mismatch!!!')
      if update_value > base_value:
        print('New dataframe exceed by', update_value-base_value, '('+str(1-update_value*100/base_value)+'%)')
      else:
        print('New dataframe exceed by', base_value-update_value, '('+str(update_value*100/base_value)+'%)')

In [ ]:
df_base = pd.read_excel('/content/drive/MyDrive/My Project/MFP_Volunteer/budget_df.xlsx', sheet_name = 'raw')
df = df_base.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52031 entries, 0 to 52030
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   REF_DOC           52031 non-null  object 
 1   REF_PAGE_NO       52031 non-null  int64  
 2   MINISTRY          52031 non-null  object 
 3   BUDGETARY_UNIT    52031 non-null  object 
 4   BUDGET_PLAN       52031 non-null  object 
 5   CROSS_FUNC?       52031 non-null  bool   
 6   PROJECT           25017 non-null  object 
 7   OUTPUT            24648 non-null  object 
 8   CATEGORY_LV1      52019 non-null  object 
 9   CATEGORY_LV2      49580 non-null  object 
 10  CATEGORY_LV3      33257 non-null  object 
 11  CATEGORY_LV4      22755 non-null  object 
 12  CATEGORY_LV5      457 non-null    object 
 13  CATEGORY_LV6      0 non-null      float64
 14  ITEM_DESCRIPTION  52029 non-null  object 
 15  FISCAL_YEAR       52031 non-null  int64  
 16  OBLIGED?          52031 non-null  bool  

In [ ]:
track_status(df, df_base, ['AMOUNT'])

Row: 52,031 and Columns: 22
AMOUNT of new df: 4,171,104,770,628.0
AMOUNT of base df: 4,171,104,770,628.0
pass: the AMOUNT are match


In [ ]:
df_moph_base = df[df['BUDGETARY_UNIT'] == 'สำนักงานปลัดกระทรวงสาธารณสุข']
df_moph = df_moph_base.copy()

In [ ]:
df_moph.reset_index(drop = True, inplace = True)

In [ ]:
track_status(df_moph, df_moph_base, ['AMOUNT'])

Row: 1,669 and Columns: 22
AMOUNT of new df: 137,883,473,400.0
AMOUNT of base df: 137,883,473,400.0
pass: the AMOUNT are match


In [ ]:
extracted_list = list(df_moph.ITEM_DESCRIPTION.str.split(" "))

In [ ]:
keep = []
for i in range(len(extracted_list)):
  for text in extracted_list[i]:
    if "โรงพยาบาล" in text:
      df_moph.loc[i, 'HOSPITAL'] = text

In [ ]:
track_status(df_moph, df_moph_base, ['AMOUNT'])

Row: 1,669 and Columns: 23
AMOUNT of new df: 137,883,473,400.0
AMOUNT of base df: 137,883,473,400.0
pass: the AMOUNT are match


In [ ]:
df_moph_bkup = df_moph.copy()

In [ ]:
fiscal_yr_max = df_moph_bkup.groupby(['ITEM_DESCRIPTION'])[['FISCAL_YEAR']].max().reset_index().rename(columns = {'FISCAL_YEAR': 'MAX_FISCAL_YEAR'})
fiscal_yr_min = df_moph_bkup.groupby(['ITEM_DESCRIPTION'])[['FISCAL_YEAR']].min().reset_index().rename(columns = {'FISCAL_YEAR': 'MIN_FISCAL_YEAR'})

In [ ]:
df_moph = df_moph.merge(fiscal_yr_max, how = 'left', on = 'ITEM_DESCRIPTION')
df_moph = df_moph.merge(fiscal_yr_min, how = 'left', on = 'ITEM_DESCRIPTION')

In [ ]:
df_moph['DURATION'] = df_moph['MAX_FISCAL_YEAR'] - df_moph['MIN_FISCAL_YEAR'] + 1

In [ ]:
track_status(df_moph, df_moph_base, ['AMOUNT'])

Row: 1,669 and Columns: 26
AMOUNT of new df: 137,883,473,400.0
AMOUNT of base df: 137,883,473,400.0
pass: the AMOUNT are match


In [ ]:
df_moph.to_excel('df_add_hospital.xlsx', sheet_name = 'raw', index = False)